# Event-Based Forecasting

In this project, I explored event-based forecasting by analyzing how breaking news and financial tweets influence stock market movements. Using a dataset containing financial news sentiment and stock price data, I examined the relationship between sentiment-driven market reactions and Cumulative Abnormal Returns (CAR).

## Loading Data

In [ ]:
import zipfile
import pandas as pd
import numpy as np
import re

import yfinance as yf
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt
from tqdm import tqdm

In [1]:
# Path to your zip file (update this with your actual path)
zip_path = 'Financial News Headlines.zip'

# Function to load and inspect each CSV file
def load_and_inspect_csv(zip_file, csv_name):
    with zip_file.open(csv_name) as f:
        df = pd.read_csv(f)
    
    print(f"\n=== {csv_name} ===")
    print(f"Shape: {df.shape}")
    print("\nFirst 5 rows:")
    print(df.head())
    print("\nInfo:")
    print(df.info())
    print("\nDescribe:")
    print(df.describe())
    print("\nData types:")
    print(df.dtypes)
    print("\nMissing values per column:")
    print(df.isnull().sum())
    
    # If there's a date column, check the date range
    date_cols = [col for col in df.columns if 'date' in col.lower()]
    if date_cols:
        df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors='coerce')
        print(f"\nDate range: {df[date_cols[0]].min()} to {df[date_cols[0]].max()}")
    
    return df

# Load and inspect each file
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # List files in the zip
    print("Files in the zip archive:")
    print(zip_file.namelist())
    
    # Load each CSV
    cnbc_df = load_and_inspect_csv(zip_file, 'cnbc_headlines.csv')
    guardian_df = load_and_inspect_csv(zip_file, 'guardian_headlines.csv')
    reuters_df = load_and_inspect_csv(zip_file, 'reuters_headlines.csv')

Files in the zip archive:
['cnbc_headlines.csv', 'guardian_headlines.csv', 'reuters_headlines.csv']

=== cnbc_headlines.csv ===
Shape: (3080, 3)

First 5 rows:
                                           Headlines  \
0  Jim Cramer: A better way to invest in the Covi...   
1     Cramer's lightning round: I would own Teradyne   
2                                                NaN   
3  Cramer's week ahead: Big week for earnings, ev...   
4  IQ Capital CEO Keith Bliss says tech and healt...   

                             Time  \
0   7:51  PM ET Fri, 17 July 2020   
1   7:33  PM ET Fri, 17 July 2020   
2                             NaN   
3   7:25  PM ET Fri, 17 July 2020   
4   4:24  PM ET Fri, 17 July 2020   

                                         Description  
0  "Mad Money" host Jim Cramer recommended buying...  
1  "Mad Money" host Jim Cramer rings the lightnin...  
2                                                NaN  
3  "We'll pay more for the earnings of the non-Co...  
4  Ke

## Cleaning the Data

Each news source has a slightly different format, so the script has separate cleaning functions.

In [2]:
def parse_guardian_date(date_str):
    """Flexible date parser for Guardian's various date formats"""
    try:
        # Try the original format (e.g., "18-Jul-20")
        return datetime.strptime(date_str, "%d-%b-%y").date()
    except ValueError:
        try:
            # Try format without century (e.g., "18-Jul-2020")
            return datetime.strptime(date_str, "%d-%b-%Y").date()
        except ValueError:
            try:
                # Try month-first format (e.g., "Jul-18")
                return datetime.strptime(date_str, "%b-%d").date().replace(year=datetime.now().year)
            except ValueError:
                # If all fails, return None (will be filtered out later)
                return None

def clean_cnbc_data(df):
    """Clean CNBC headlines data"""
    df = df.copy()
    
    # Drop rows with all NA values
    df = df.dropna(how='all')
    
    # Parse datetime with multiple format attempts
    df['datetime'] = pd.to_datetime(
        df['Time'],
        errors='coerce',
        format='%I:%M %p ET %a, %d %B %Y'
    )
    
    # For rows where parsing failed, try alternative formats
    mask = df['datetime'].isna()
    if mask.any():
        # Try format without ET
        df.loc[mask, 'datetime'] = pd.to_datetime(
            df.loc[mask, 'Time'].str.replace(' ET', ''),
            errors='coerce',
            format='%I:%M %p %a, %d %B %Y'
        )
    
    # Drop rows where datetime couldn't be parsed
    df = df.dropna(subset=['datetime'])
    
    # Create date column
    df['date'] = df['datetime'].dt.date
    
    # Clean text columns
    text_cols = ['Headlines', 'Description']
    for col in text_cols:
        df[col] = df[col].fillna('').astype(str)
        df[col] = df[col].str.strip()
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
    
    return df.drop(columns=['Time', 'datetime'])

def clean_guardian_data(df):
    """Clean Guardian headlines data with robust date parsing"""
    df = df.copy()
    
    # Apply flexible date parser
    df['date'] = df['Time'].apply(parse_guardian_date)
    
    # Drop rows where date couldn't be parsed
    df = df.dropna(subset=['date'])
    
    # Clean headlines
    df['Headlines'] = df['Headlines'].astype(str).str.strip()
    df['Headlines'] = df['Headlines'].str.replace(r'\s+', ' ', regex=True)
    df['Headlines'] = df['Headlines'].str.replace(r'\n', ' ', regex=True)
    
    return df.drop(columns=['Time'])

def clean_reuters_data(df):
    """Clean Reuters headlines data"""
    df = df.copy()
    
    # Convert Time to datetime
    df['date'] = pd.to_datetime(
        df['Time'],
        format='%b %d %Y',
        errors='coerce'
    ).dt.date
    
    # Drop rows where date couldn't be parsed
    df = df.dropna(subset=['date'])
    
    # Clean text columns
    text_cols = ['Headlines', 'Description']
    for col in text_cols:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
    
    return df.drop(columns=['Time'])

def standardize_dataframes(cnbc, guardian, reuters):
    """Standardize all dataframes to common format"""
    cnbc = cnbc.copy()
    guardian = guardian.copy()
    reuters = reuters.copy()
    
    # Create consistent column names and structure
    cnbc = cnbc[['date', 'Headlines', 'Description']]
    guardian = guardian[['date', 'Headlines']]
    reuters = reuters[['date', 'Headlines', 'Description']]
    
    # Add source identifier
    cnbc['source'] = 'CNBC'
    guardian['source'] = 'Guardian'
    reuters['source'] = 'Reuters'
    
    # For Guardian which lacks Description, create empty column
    guardian['Description'] = ''
    
    # Ensure consistent order
    final_cols = ['date', 'source', 'Headlines', 'Description']
    return (
        cnbc[final_cols],
        guardian[final_cols],
        reuters[final_cols]
    )

def merge_datasets(cnbc, guardian, reuters):
    """Merge all datasets into one unified dataframe"""
    return pd.concat([cnbc, guardian, reuters], ignore_index=True)

# Load your original data
# Clean each dataset
cnbc_clean = clean_cnbc_data(cnbc_df)
guardian_clean = clean_guardian_data(guardian_df)
reuters_clean = clean_reuters_data(reuters_df)

# Standardize formats
cnbc_std, guardian_std, reuters_std = standardize_dataframes(cnbc_clean, guardian_clean, reuters_clean)

# Merge datasets
all_news = merge_datasets(cnbc_std, guardian_std, reuters_std)

# Final cleaning on merged data
all_news = all_news.sort_values('date')
all_news = all_news.drop_duplicates(subset=['date', 'source', 'Headlines'])

print("\nCleaned data overview:")
print(f"Total articles: {len(all_news)}")
print(f"Date range: {all_news['date'].min()} to {all_news['date'].max()}")
print("\nSample of cleaned data:")
print(all_news.head())


Cleaned data overview:
Total articles: 50610
Date range: 2017-12-17 to 2025-07-18

Sample of cleaned data:
             date    source  \
17900  2017-12-17  Guardian   
17899  2017-12-17  Guardian   
17901  2017-12-17  Guardian   
17902  2017-12-17  Guardian   
17903  2017-12-17  Guardian   

                                               Headlines Description  
17900  Why business could prosper under a Corbyn gove...              
17899  Peter Preston on press and broadcasting Paul D...              
17901  Youngest staff to be given UK workplace pensio...              
17902  Grogonomics This year has been about companies...              
17903  Butchers carve out a niche as UK shoppers opt ...              


In [3]:
# Check date parsing success rate
print(f"CNBC kept {len(cnbc_clean)}/{len(cnbc_df)} rows ({len(cnbc_clean)/len(cnbc_df):.1%})")
print(f"Guardian kept {len(guardian_clean)}/{len(guardian_df)} rows ({len(guardian_clean)/len(guardian_df):.1%})")
print(f"Reuters kept {len(reuters_clean)}/{len(reuters_df)} rows ({len(reuters_clean)/len(reuters_df):.1%})")

# Check for remaining issues
print("\nDate ranges:")
print("CNBC:", cnbc_clean['date'].min(), "to", cnbc_clean['date'].max())
print("Guardian:", guardian_clean['date'].min(), "to", guardian_clean['date'].max())
print("Reuters:", reuters_clean['date'].min(), "to", reuters_clean['date'].max())

# Check final merged data
print("\nMerged data info:")
print(all_news.info())
print("\nMissing values:")
print(all_news.isnull().sum())

CNBC kept 119/3080 rows (3.9%)
Guardian kept 17800/17800 rows (100.0%)
Reuters kept 32770/32770 rows (100.0%)

Date ranges:
CNBC: 2018-03-05 to 2020-07-02
Guardian: 2017-12-17 to 2025-07-18
Reuters: 2018-03-20 to 2020-07-18

Merged data info:
<class 'pandas.core.frame.DataFrame'>
Index: 50610 entries, 17900 to 14040
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         50610 non-null  object
 1   source       50610 non-null  object
 2   Headlines    50610 non-null  object
 3   Description  50610 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
None

Missing values:
date           0
source         0
Headlines      0
Description    0
dtype: int64


In [4]:
# CNBC

def clean_cnbc_data(df):
    """Improved CNBC headlines data cleaner"""
    df = df.copy()
    df = df.dropna(how='all')
    
    # Initialize datetime column
    df['datetime'] = pd.NaT
    
    # List of possible CNBC date formats to try
    date_formats = [
        '%I:%M %p ET %a, %d %B %Y',  # "7:51 PM ET Fri, 17 July 2020"
        '%I:%M %p %a, %d %B %Y',      # "7:51 PM Fri, 17 July 2020" (no ET)
        '%a, %d %B %Y %I:%M %p ET',   # "Fri, 17 July 2020 7:51 PM ET"
        '%B %d, %Y %I:%M %p ET',      # "July 17, 2020 7:51 PM ET"
        '%d-%b-%y %I:%M %p ET',       # "17-Jul-20 7:51 PM ET"
    ]
    
    # Try each format until we find one that works
    for fmt in date_formats:
        mask = df['datetime'].isna()
        if not mask.any():
            break
        df.loc[mask, 'datetime'] = pd.to_datetime(
            df.loc[mask, 'Time'],
            errors='coerce',
            format=fmt
        )
    
    # For remaining NA values, try parsing without time
    mask = df['datetime'].isna()
    if mask.any():
        df.loc[mask, 'datetime'] = pd.to_datetime(
            df.loc[mask, 'Time'],
            errors='coerce'
        )
    
    # Drop rows where datetime couldn't be parsed
    df = df.dropna(subset=['datetime'])
    df['date'] = df['datetime'].dt.date
    
    # Clean text columns
    text_cols = ['Headlines', 'Description']
    for col in text_cols:
        df[col] = df[col].fillna('').astype(str)
        df[col] = df[col].str.strip()
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
    
    return df.drop(columns=['Time', 'datetime'])

def fix_guardian_dates(df):
    """Ensure Guardian dates are properly parsed (fix future dates)"""
    df = df.copy()
    current_year = datetime.now().year
    
    # Convert dates to datetime if they're strings
    if df['date'].dtype == object:
        df['date'] = pd.to_datetime(df['date']).dt.date
    
    # Fix dates in the future (likely 2-digit year misinterpretation)
    future_mask = df['date'].apply(lambda x: x.year > current_year)
    df.loc[future_mask, 'date'] = df.loc[future_mask, 'date'].apply(
        lambda x: x.replace(year=x.year - 100) if x.year > current_year else x
    )
    
    return df

## Event-Based Forecasting

Linking News Events to Stock Market Movements

In [8]:
# Identify the Correct Price Column
def get_price_column(stock_data):
    """Safely identify the price column"""
    for col in ['Adj Close', 'Close', 'Adj_Close', 'adjclose']:
        if col in stock_data.columns:
            return col
    return stock_data.columns[0]

# Match News Data to Trading Days
def match_news_to_trading_days(news_data, stock_ticker='SPY'):
    """Enhanced matching with error handling"""
    news = news_data.copy()
    news['date'] = pd.to_datetime(news['date'], errors='coerce')
    news = news.dropna(subset=['date'])
    
    # Download data with verification
    start = news['date'].min() - timedelta(days=90)
    end = min(news['date'].max() + timedelta(days=90), datetime.now())
    stock = yf.download(stock_ticker, start=start, end=end, progress=False)
    stock.index = pd.to_datetime(stock.index).normalize()
    
    price_col = get_price_column(stock)
    print(f"Using price column: '{price_col}'")
    
    # Business day adjustment
    news['adj_date'] = news['date'].apply(lambda x: x + BDay(0) if x.weekday() > 4 else x)
    trading_days = pd.Series(stock.index.unique()).sort_values()
    
    def find_nearest_trading_day(date):
        time_diff = abs(trading_days - date)
        return trading_days[time_diff.idxmin()]
    
    news['trade_date'] = news['adj_date'].apply(find_nearest_trading_day)
    news['gap_days'] = (news['trade_date'] - news['date']).dt.days
    
    return news, stock, price_col

# Calculate Cumulative Abnormal Returns (CAR)
def calculate_car(stock_data, news_matches, price_col, window=(-3,3)):
    """Robust CAR calculation with sentiment handling"""
    results = []
    
    for trade_date, group in tqdm(news_matches.groupby('trade_date'),
                                 desc="Processing events"):
        try:
            # Get price window
            prices = stock_data[price_col].loc[
                trade_date + timedelta(days=window[0]) : 
                trade_date + timedelta(days=window[1])
            ]
            if len(prices) < 2:
                continue
                
            # Calculate returns
            returns = prices.pct_change().dropna()
            baseline = stock_data[price_col].pct_change().mean()
            car = float((returns - baseline).sum())  # Force conversion to float
            
            # Safely determine sentiment
            sentiment_counts = group['sentiment'].value_counts()
            if not sentiment_counts.empty:
                sentiment = sentiment_counts.idxmax()
            else:
                sentiment = 'neutral'
            
            results.append({
                'trade_date': trade_date,
                'sentiment': sentiment,
                'car': car,
                'news_count': len(group),
                'price_col': price_col  # For verification
            })
        except Exception as e:
            continue
            
    return pd.DataFrame(results)

# Main Execution (if run as a script)
if __name__ == "__main__":
    try:
        print("Loading data...")
        raw_news = pd.read_csv('news_with_sentiment.csv')
        
        print("Matching dates...")
        news_matches, stock_data, price_col = match_news_to_trading_days(raw_news)
        
        # Filter for quality matches
        filtered_news = news_matches[abs(news_matches['gap_days']) <= 2]
        print(f"\nFound {len(filtered_news)} valid events")
        print(f"Unique trading days: {filtered_news['trade_date'].nunique()}")
        
        # Calculate CAR
        print("\nCalculating CAR...")
        car_results = calculate_car(stock_data, filtered_news, price_col)
        
        if not car_results.empty:
            car_results.to_csv('car_results.csv', index=False)
            print("\nAnalysis successful!")
            print("\nTop Bullish Events:")
            print(car_results[car_results['sentiment'] == 'bullish'].sort_values('car', ascending=False).head())
            print("\nTop Bearish Events:")
            print(car_results[car_results['sentiment'] == 'bearish'].sort_values('car').head())
        else:
            print("\nNo valid CAR results calculated")
            
    except Exception as e:
        print(f"\nFatal error: {str(e)}")
        print("\nDebugging tips:")
        print("1. Check news data format:")
        print(raw_news[['date', 'sentiment']].head())
        print("\n2. Verify stock data:")
        print(stock_data.head())

Loading data...
Matching dates...
Using price column: 'Close'

Found 50393 valid events
Unique trading days: 646

Calculating CAR...


Processing events:   0%|          | 0/646 [00:00<?, ?it/s]C:\Users\milto\AppData\Local\Temp\ipykernel_20824\1639327520.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  car = float((returns - baseline).sum())  # Force conversion to float
C:\Users\milto\AppData\Local\Temp\ipykernel_20824\1639327520.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  car = float((returns - baseline).sum())  # Force conversion to float
C:\Users\milto\AppData\Local\Temp\ipykernel_20824\1639327520.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  car = float((returns - baseline).sum())  # Force conversion to float
C:\Users\milto\AppData\Local\Temp\ipykernel_20824\1639327520.py:57: FutureWarning: Calling float


Analysis successful!

Top Bullish Events:
    trade_date sentiment       car  news_count price_col
253 2018-12-27   bullish  0.055235          46     Close
574 2020-04-07   bullish  0.048128          70     Close
576 2020-04-09   bullish  0.048128         145     Close
615 2020-06-05   bullish  0.046162          63     Close
362 2019-06-05   bullish  0.044642          93     Close

Top Bearish Events:
    trade_date sentiment       car  news_count price_col
553 2020-03-09   bearish -0.173011         129     Close
557 2020-03-13   bearish -0.170600         118     Close
562 2020-03-20   bearish -0.119407         137     Close
558 2020-03-16   bearish -0.106176         190     Close
552 2020-03-06   bearish -0.088071         103     Close


### Key Takeaways

- CAR Calculation Worked – The script successfully analyzed how news sentiment correlates with stock movements.
- Bullish Events Led to Positive CAR – Certain dates had strong market gains following bullish news.
- Bearish Events Aligned with Market Crashes – Major market drops were linked to bearish news, especially during the COVID-19 crash.
- News Volume Matters – Each event had a news_count, indicating multiple articles influenced the sentiment.

## Conclusion

In this project, I explored event-based forecasting by analyzing how breaking news and financial tweets influence stock market movements. Using a dataset containing financial news sentiment and stock price data, I examined the relationship between sentiment-driven market reactions and Cumulative Abnormal Returns (CAR).

### Key Findings:

News sentiment has a measurable impact on stock prices.
- Bullish news events correlated with positive CAR values, indicating strong upward price movements.
- Bearish news events coincided with negative CAR values, aligning with market downturns.

Significant market reactions occurred during critical financial periods.
- March 2020, during the COVID-19 crash, showed extreme negative CAR values, reflecting widespread market fear and volatility.
- Conversely, periods of economic recovery saw bullish news leading to positive CAR values.

High-volume news days amplified stock movements.
- The number of news articles (news_count) was often higher on volatile trading days, reinforcing the impact of media coverage on investor sentiment.